В качестве генератора случайных чисел выберем когруэнтный метод из прошлой лаб раб 

In [10]:
import math
def congruent_method_generator():
    m =10**10
    x=get_x0(m)
    a=197
    while True:
        x = a*x%m
        yield x * 1.0 / m


def get_x0(m):
    for i in range(m-1,1,-1):
        if math.gcd(i,m) == 1:
            return i
    return 1

In [8]:
class RandomEventSimulator:

    def __init__(self):
        self._generator = congruent_method_generator()
    
    def check_event(self,p):
        _p = next(self._generator)
        return _p < p
    

In [11]:
p_list = [0.5, 0.1, 0.8]
N_SAMPLES = 10**6
simulator = RandomEventSimulator()


for p in p_list:
    answers = list()
    for i in range(N_SAMPLES):
        answers.append(simulator.check_event(p))
    happend_event_count = sum(answers)
    print(f"p: {p} happend events count :{happend_event_count}")


p: 0.5 happend events count :500247
p: 0.1 happend events count :100224
p: 0.8 happend events count :800504


In [17]:
import numpy as np
class NotSimpleEventSimulator():

    def __init__(self):
        self._generator = congruent_method_generator()
    
    def _first(self,it,condition):
        return next(x for x in it if condition(x))
    def check_event(self,k):
        p = next(self._generator)
        _cumsum = enumerate(np.cumsum(k))
        success_event = self._first(_cumsum, lambda x : p <=x[1])[0]
        bool_vector = [False for _ in k]
        bool_vector[success_event] = True
        return bool_vector

N_SAMPLES = 10**6

k_list=[
    [0.1,0.2,0.3,0.4],
    [0.8,0.1,0.05,0.05],
    [0.5,0.5]
]

simulator = NotSimpleEventSimulator()

for k in k_list:
    answers = list()
    for i in range(N_SAMPLES):
        answers.append(simulator.check_event(k))
    event_freq = [0 for i in k]
    for answer in answers:
        for i in range(len(k)):
            event_freq[i]+= answer[i]
    print(f"event freq:{event_freq}")



event freq:[100792, 200230, 298794, 400184]
event freq:[799771, 100168, 50018, 50043]
event freq:[499764, 500236]


In [24]:
class DependentEventGenerator:

    def __init__(self):
        self.generator = congruent_method_generator()
    
    def A_B(self,x1,x2,p_a,p_b_a):
        return x1<= p_a and x2<=p_b_a
    def A_not_B(self,x1,x2,p_a,p_b_a):
        return x1<= p_a and x2 > p_b_a
    def not_A_B(self,x1,x2,p_a,p_b_a):
        return x1> p_a and x2<= p_b_a
    def not_A_not_B(self,x1,x2,p_a,p_b_a):
        return x1 > p_a and x2> p_b_a
    
    def check_event(self,p_a,p_b_a):
        x1= next(self.generator)
        x2 = next(self.generator)
        p_b_not_a = 1 - p_b_a

        a_b = self.A_B(x1,x2,p_a,p_b_a)
        a_not_b = self.A_not_B(x1,x2,p_a,p_b_a)
        not_a_b = self.not_A_B(x1,x2,p_a,p_b_not_a)
        not_a_not_b = self.not_A_not_B(x1,x2,p_a,p_b_not_a)

        return [a_b,a_not_b,not_a_b,not_a_not_b]

def find_p_b(p_b_a,p_a):
    return p_b_a* p_a +(1-p_b_a)*(1-p_a)

def find_p_not_b_a(p_b_a,p_a):
    p_b = find_p_b(p_b_a,p_a)
    print(f"p_a:{p_a}, p_b_a: {p_b_a}, p_b:{p_b}")
    p_a_not_b = (p_a-1+p_b)/(2*p_b -1)
    return p_a_not_b*(1-p_b)/p_a

N_SAMPLES = 10**6
p_a= [0.1,0.4,0.3]
p_b_a =[0.2,0.1,0.4]
k = len(p_a)

p_ab = [p_a[i]*p_b_a[i] for i in range(k)]
p_a_not_b = [p_a[i]*(1-p_b_a[i]) for i in range(k)]
p_not_a_b =[(1- p_a[i])*find_p_not_b_a(p_b_a[i],p_a[i]) for i in range(k)]
p_not_a_not_b =[(1-p_a[i])*(1-p_not_a_b[i]) for i in range(k)]

simulator = DependentEventGenerator()

for i in range(k):
    answers = list()
    for j in range(N_SAMPLES):
        answers.append(simulator.check_event(p_a[i],p_b_a[i]))
    
    freq = [0 for _ in answers[0]]
    for answer in answers:
        for _i in range(len(answer)):
            freq[_i]+=answer[_i]
    
    print([p_ab[i],p_a_not_b[i],p_not_a_b[i],p_not_a_not_b[i]])
    print(freq)






p_a:0.1, p_b_a: 0.2, p_b:0.7400000000000001
p_a:0.4, p_b_a: 0.1, p_b:0.5800000000000001
p_a:0.3, p_b_a: 0.4, p_b:0.54


IndexError: list index out of range